# Automated ML

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Udacity-exp-p3'

experiment=Experiment(ws, experiment_name)

In [3]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Heart Disease"
description_text = "Heart Disease"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [4]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [15]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "AML-Compute"
# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 3)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded.......................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

This creates a general AutoML settings object. These inputs must match what was used when training in the portal. label_column_name has to be y for example.

In [16]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric": 'accuracy'
}


automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="churn",   
                             #path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [17]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


ValidationException: ValidationException:
	Message: Validation error(s): [{
    "additional_properties": {
        "debugInfo": null
    },
    "code": "UserError",
    "severity": 2,
    "message": "Y is empty",
    "message_format": "{dataName} is empty",
    "message_parameters": {
        "dataName": "Y"
    },
    "reference_code": null,
    "details_uri": null,
    "target": "training_data",
    "details": [
        {
            "additional_properties": {
                "debugInfo": null
            },
            "code": null,
            "severity": null,
            "message": "null",
            "message_format": null,
            "message_parameters": {},
            "reference_code": null,
            "details_uri": null,
            "target": null,
            "details": [],
            "inner_error": null
        }
    ],
    "inner_error": {
        "additional_properties": {},
        "code": "BadData",
        "inner_error": {
            "additional_properties": {},

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:

from azureml.widgets import RunDetails
RunDetails(remote_run).show()

NameError: name 'remote_run' is not defined

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run_aml, fitted_model_aml = remote_run.get_output()
model_name = best_run_aml.properties['model_name']
print(best_run_aml)

In [ ]:
print(fitted_model_aml[-1])

In [ ]:
#TODO: Save the best model
best_run_aml.download_file('outputs/model.pkl', 'outputs/automl.pkl')
best_run_aml.download_file('outputs/scoring_file_v_1_0_0.py', 'outputs/score_aml.py')
best_run_aml.download_file('automl_driver.py', 'outputs/automl_driver.py')


In [ ]:
import pickle
file = open("./automl.pkl",'rb')
aml_model = pickle.load(file)
file.close()

##filename = 'best_automl_model.sav'
##  joblib.dump(fitted,filename)

In [ ]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['churn'])]

y_test = df_test['churn']
X_test = df_test.drop(['churn'], axis=1)

In [ ]:
print(accuracy_score(aml_model.predict(X_test, y_test))

In [ ]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

In [ ]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Register the model
import sklearn

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model_name = best_run.properties['model_name']

run_id = best_run.id
experiment_name = best_run.experiment.name

model = Model.register(workspace=ws,
                       model_name=model_name, 
                       model_path='inference/model.pkl',
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version=sklearn.__version__,
                       description='Auto ML model to predict churn in telecommunications.',
                       tags={'area': 'churn', 'type': 'classification'})

print('Name:', model.name)
print('Version:', model.version)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:

# create inference configuration
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

myenv = Environment.from_conda_specification(name="myenv", file_path="inference/conda_env.yml")
inference_config = InferenceConfig(entry_script="inference/score.py", environment=myenv)

# display the environment file
with open('inference/conda_env.yml', 'r') as file:
    env_file = file.read()
    print(env_file)

In [ ]:
from azureml.core.webservice import AciWebservice

# define deployment configuration
aci_deployment_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                           memory_gb=1,
                                                           tags={'area': "heart-failure", 'type': "classification"},
                                                           description="Predict heart failure mortality using classification model",
                                                           enable_app_insights=True)

# deploy model as webservice using Azure Container Instance(ACI)
aci_service_name = "aci-heart-failure-deploy"

aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_deployment_config, overwrite=True)
aci_service.wait_for_deployment(show_output=True)

print(aci_service.state)

In [ ]:

# get the active api endpoint for scoring
print(aci_service.scoring_uri)

In [ ]:

import json

test_ds1 = test_ds.drop('churn',axis=1)
data= ({'data':test_ds1[0:3].to_dict(orient='records'),'method':'predict'})
test_sample=json.dumps(data)
#test_sampole = json.dumps({'data': test_ds[0:2].tolist(),'method': 'predict'})
output= service.run(test_sample)
print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
data = {"data":
        [
          {
            "gender": 68,
            "SeniorCitizen": 1,
            "Partner":Yes ,
            "Dependents": No,
            "tenure": 1,
            "PhoneService": No,
            "MultipleLines": No,
            "InternetService": DSL,
            "OnlineSecurity": No,
            "OnlineBackup": Yes,
            "DeviceProtection": No,
            "TechSupport": No,
            "StreamingTV": No,
            "StreamingMovies": No,
            "Contract":Month-to-month,
            "PaperlessBilling":Yes,
            "PaymentMethod":Electronic check,
            "MonthlyCharges":29.85,
            "TotalCharges":29.85
            
          }
      ]}#,'method':'predict'}
test_sample = json.dumps(data)
output= service.run(test_sample)
print(output)

In [ ]:
import requests
scoring_uri='http://212f2021-c0d2-494a-b284-426d0a8b6722.southcentralus.azurecontainer.io/score'
key=''
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'
data = {"data":
        [
          {
            "gender": 68,
            "SeniorCitizen": 1,
            "Partner":Yes ,
            "Dependents": No,
            "tenure": 1,
            "PhoneService": No,
            "MultipleLines": No,
            "InternetService": DSL,
            "OnlineSecurity": No,
            "OnlineBackup": Yes,
            "DeviceProtection": No,
            "TechSupport": No,
            "StreamingTV": No,
            "StreamingMovies": No,
            "Contract":Month-to-month,
            "PaperlessBilling":Yes,
            "PaymentMethod":Electronic check,
            "MonthlyCharges":29.85,
            "TotalCharges":29.85
            
          }
      ]
    }
test_sample = json.dumps(data)


resp = requests.post(scoring_uri, test_sample, headers=headers)
print(resp.json())

In [ ]:
service.delete()
print("service deleted")

In [ ]:
aml_compute.delete()
print("compute deleted")